# Data collection chain

This goal of this notebook is to trace a data acquisition call all the way from the high-level function down to the packet level, and to determine how the data is arranged.

`baseband.py`  
`RoachBaseband.get_data(self, nread=2, demod=True)` calls `self.get_data_udp(self, nread=2, demod=True)`, which is inherited from `RoachInterface`.

`interface.py`  
`RoachInterface.get_data_udp(self, n_read=2, demod=True)` figures out the number of channels `nch` and sets `    chan_offset = 1`. The key call is  

    data, seqnos = udp_catcher.get_udp_data(self, npkts=nread * 16 * nch, streamid=1,
        chans=self.fpga_fft_readout_indexes + chan_offset,
        nfft=self.nfft, addr=(self.host_ip, 12345))

which returns arrays `data` and `seqnos`. Note that `chans` uses the fpga indexing with a fixed offset of 1.

`udp_catcher.py`  
  `get_udp_data()` calls `get_udp_packets()` then decodes them by calling `decode_packets()` and returns them.  
  `get_udp_packets()` seems to straightforwardly return a list of received packets in chronological order.  
  `decode_packets()` does some low-level parsing of the packets and returns arrays `darray` and `seqnos`. The data array is re-arranged right at the end using `np.roll`, and I don't understand what's going on here.